<a href="https://colab.research.google.com/github/moheb432/principles-of-smile/blob/main/Copy_of_k_fold_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.models import Sequential
from keras.layers.convolutional import ZeroPadding2D
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix


In [ ]:
!git clone "https://github.com/moheb432/principles-of-smile.git"

Cloning into 'principles-of-smile'...
remote: Enumerating objects: 6536, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 6536 (delta 4), reused 0 (delta 0), pack-reused 6520
Receiving objects: 100% (6536/6536), 342.68 MiB | 30.24 MiB/s, done.
Resolving deltas: 100% (323/323), done.
Checking out files: 100% (6127/6127), done.


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Biometrics_karin/dataset_2_.csv")
df


,img,9-gummy,16-Incisal Embrasure,4-color,41-Central Incisor W/H Ratio,18-black triangle,8-gaps,52-Crooked,chopped,crooked,chopped_and_half,missing
0,41001_B.jpg,1,1,1,1,0,0,0,0,0,0,0
1,41002_B.jpg,0,1,1,1,0,1,0,0,0,0,0
2,41003_B.jpg,1,1,0,1,0,0,0,0,0,0,0
3,41004_B.jpg,1,1,1,1,0,1,0,0,0,0,0
4,41005_B.jpg,1,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2458,aug_(e30)_0_8203.jpg,1,1,1,1,0,0,0,1,0,1,1
2459,aug_(e30)_0_8739.jpg,1,1,1,1,0,0,0,1,0,1,1
2460,aug_(e30-a)_0_1272.jpg,0,1,0,0,0,0,0,0,0,0,0
2461,aug_(e30-a)_0_2605.jpg,0,1,0,0,0,0,0,0,0,0,0


In [ ]:

missing = df.isnull().sum()
missing


img                             0
9-gummy                         0
16-Incisal Embrasure            0
4-color                         0
41-Central Incisor W/H Ratio    0
18-black triangle               0
8-gaps                          0
52-Crooked                      0
chopped                         0
crooked                         0
chopped_and_half                0
missing                         0
dtype: int64

In [ ]:
classes = ['9-gummy','16-Incisal Embrasure','4-color',	'41-Central Incisor W/H Ratio',	'18-black triangle',	'8-gaps',	'52-Crooked',	'chopped',	'crooked',	'chopped_and_half',	'missing']
def append_ext(fn):
    return str(fn)+".jpg"

#loading dataset labels from csv 
# df = pd.read_csv("/content/principles-of-smile/dataset_and_augmented.csv")

# df["img"] = df["img"].apply(append_ext)

df = df.replace(np.nan, 0)
classes = ['9-gummy','16-Incisal Embrasure','4-color',	'41-Central Incisor W/H Ratio',	'18-black triangle',	'8-gaps',	'52-Crooked',	'chopped',	'crooked',	'chopped_and_half',	'missing']
df


,img,9-gummy,16-Incisal Embrasure,4-color,41-Central Incisor W/H Ratio,18-black triangle,8-gaps,52-Crooked,chopped,crooked,chopped_and_half,missing
0,41001_B.jpg,1,1,1,1,0,0,0,0,0,0,0
1,41002_B.jpg,0,1,1,1,0,1,0,0,0,0,0
2,41003_B.jpg,1,1,0,1,0,0,0,0,0,0,0
3,41004_B.jpg,1,1,1,1,0,1,0,0,0,0,0
4,41005_B.jpg,1,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2458,aug_(e30)_0_8203.jpg,1,1,1,1,0,0,0,1,0,1,1
2459,aug_(e30)_0_8739.jpg,1,1,1,1,0,0,0,1,0,1,1
2460,aug_(e30-a)_0_1272.jpg,0,1,0,0,0,0,0,0,0,0,0
2461,aug_(e30-a)_0_2605.jpg,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='val_binary_accuracy',patience=10,mode='max',restore_best_weights=True)
def MODEL():
  model = Sequential()
  model.add(keras.Input(shape=(224,224,3)))
  model.add(Conv2D(64, (3, 3), activation='relu',name="conv1_1"))
  model.add(Conv2D(64, (3, 3), activation='relu',name="conv1_2"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.add(Conv2D(128, (3, 3), activation='relu',name="conv2_1"))
  model.add(Conv2D(128, (3, 3), activation='relu',name="conv2_2"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.add(Conv2D(256, (3, 3), activation='relu',name="conv3_1"))
  model.add(Conv2D(256, (3, 3), activation='relu',name="conv3_2"))
  model.add(Conv2D(256, (3, 3), activation='relu',name="conv3_3"))
  model.add(MaxPool2D((2,2), strides=(2,2)))
  
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv4_1"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv4_2"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv4_3"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.add(Conv2D(512, (3, 3), activation='relu',name="conv5_1"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv5_2"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv5_3"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.load_weights('/content/drive/MyDrive/Biometrics_karin/vgg_ratio_60-15-25_weights.h5', by_name=True,skip_mismatch=True)

  #freezing layers
  for layer in model.layers:
      layer.trainable = False

  model.get_layer("conv3_1").trainable =True
  #model.get_layer("conv3_3").trainable =True
  #model.get_layer("conv4_1").trainable =True
  model.get_layer("conv4_1").trainable =True
  model.get_layer("conv4_2").trainable =True
  model.get_layer("conv5_3").trainable =True
  model.get_layer("conv5_1").trainable =True


  #adding output layers
  model.add(Flatten())

  model.add( Dense(64,use_bias=False, kernel_regularizer=keras.regularizers.L2(0.02)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(Dropout(0.5))

  model.add( Dense(16, use_bias=False,kernel_regularizer=keras.regularizers.L2(0.015)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(Dense(11, activation='sigmoid'))
  # summarize
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss = 'BinaryCrossentropy',metrics=['binary_accuracy'])
  #fit model 
  # history=model.fit(x=training_set, validation_data=val_set, epochs=30 ,callbacks=[es]) 
  # model.summary()
  return model


In [ ]:
pip install scikeras


In [ ]:
from sklearn.model_selection import KFold
import scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

train_datagen = ImageDataGenerator(samplewise_center=True,rescale=2./255)
test_datagen = ImageDataGenerator(samplewise_center=True,rescale=2./255) 

training_set = train_datagen.flow_from_dataframe(dataframe=df,
                                                        directory="/content/principles-of-smile/K-fold/train",
                                                        x_col="img",
                                                        y_col=classes,
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode="raw",seed=0)

test_set = test_datagen.flow_from_dataframe(dataframe=df,
                                                        directory="/content/principles-of-smile/K-fold/test",
                                                        x_col="img",
                                                        y_col=classes,
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode="raw",seed=0, shuffle = False)

# Store the data in X_test, y_test variables by iterating over the batches
batch_size =1
X_test, y_test = next(test_set)
for i in range(int(len(test_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(test_set)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)

print(X_test.shape, y_test.shape)

# train 
batch_size =1
X_train, y_train = next(training_set)
for i in range(int(len(training_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(training_set)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)

print(X_train.shape, y_train.shape)





/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1947 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2371 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 516 validated image filenames.
Found 92 validated image filenames.
(92, 224, 224, 3) (92, 11)
(516, 224, 224, 3) (516, 11)


In [ ]:
estimator = KerasClassifier(build_fn=MODEL, epochs=20, batch_size=5, verbose=0)
kf = KFold(n_splits = 5, shuffle = True, random_state = None)
results = cross_val_score(estimator,X_train, y_train, cv=kf)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

Baseline: 16.47% (3.72%)
